# 080. Simple RNN

- Simple RNN을 이용한 **시계열 예측**

<img src='time_prediction.png' width=40% />

In [1]:
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt

## 시계열 Toy Data 생성 

- sequence toy data로 sine wave 생성 (predictable 한 data point 생성)  
- RNN은 시계열 Data 생성이 까다로움

In [2]:
numbers = [[i] for i in range(105)]
numbers[:5]

[[0], [1], [2], [3], [4]]

- input, label data 생성

In [3]:
data = []
target = []

for i in range(5, len(numbers)):
    data.append(numbers[i-5:i])
    target.append(numbers[i][0] * 2)

data = torch.tensor(data, dtype=torch.float) / 100
target = torch.tensor(target, dtype=torch.float).unsqueeze(1) / 100
print(data[10])
print(target[10])

tensor([[0.1000],
        [0.1100],
        [0.1200],
        [0.1300],
        [0.1400]])
tensor([0.3000])


In [4]:
data.shape, target.shape

(torch.Size([100, 5, 1]), torch.Size([100, 1]))

## Define the RNN

- `nn.RNN`을 사용하여 RNN 레이어를 생성한 다음, 원하는 출력 크기를 얻기 위해 fully-connected layer를 추가  

- RNN의 parameters:  

    * **input_size** - input X의 feature 수
    * **hidden_dim** - RNN 출력 및 hidden state의 neuron 수
    * **n_layers** - RNN을 구성하는 레이어 수 (일반적으로 1-3 개) 1보다 크면 stacked RNN을 생성한다는 의미
    * **batch_first** - RNN의 입력 / 출력이 첫 번째 차원으로 batch_size를 가질지 여부 (batch_size, seq_length, hidden_dim)

In [5]:
class RNN(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers=1):
        super(RNN, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, 
                          batch_first=True, nonlinearity='relu')
        self.fc  = nn.Linear(hidden_dim, output_size)
        
    def forward(self, x, hidden):
        # x (batch_size, seq_length, input_size)
        # hidden (n_layers, batch_size, hidden_dim)
        # r_out (batch_size, time_step, hidden_size)
        r_out, hidden = self.rnn(x, hidden)
        
        # sequence-to-vector (last time step만 사용)
        output = self.fc(r_out[:, -1, :])
        
        return output, hidden

### Check the input and output dimensions

- 모델이 예상대로 작동하는지 확인

In [6]:
# test that dimensions 
test_rnn = RNN(input_size=1, output_size=1, hidden_dim=10)

# test out rnn sizes
test_out, test_h = test_rnn(data, None)  #None - hidden 초기화

print('Output size: ', test_out.size())
print('Hidden state size: ', test_h.size())

Output size:  torch.Size([100, 1])
Hidden state size:  torch.Size([1, 100, 10])


---
## Training the RNN

- 지정된 하이퍼 파라미터를 사용하여 RNN을 인스턴스화 및 train

In [7]:
# hyperparameters
input_size=1 
output_size=1
hidden_dim=32

# instantiate an RNN
rnn = RNN(input_size, output_size, hidden_dim)
print(rnn)

RNN(
  (rnn): RNN(1, 32, batch_first=True)
  (fc): Linear(in_features=32, out_features=1, bias=True)
)


In [8]:
# MSE loss and Adam optimizer with a learning rate of 0.01
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.01) 

In [9]:
n_steps = 1000
print_every = 100

# initialize the hidden state
hidden = None      

for step in range(n_steps):

    prediction, hidden = rnn(data, hidden)

    ## Representing Memory ##
    # make a new variable for hidden and detach the hidden state from its history
    # this way, we don't backpropagate through the entire history
    hidden = hidden.data

    # calculate the loss
    loss = criterion(prediction, target)
    # zero gradients
    optimizer.zero_grad()
    # perform backprop and update weights
    loss.backward()
    optimizer.step()

    # display loss and predictions
    if step % print_every == 0:        
        print('Loss after step {1} = {0} '.format(loss.item(),  step))

Loss after step 0 = 1.1827688217163086 
Loss after step 100 = 0.0523453988134861 
Loss after step 200 = 0.051971014589071274 
Loss after step 300 = 0.04098355770111084 
Loss after step 400 = 0.0013526598922908306 
Loss after step 500 = 0.033330272883176804 
Loss after step 600 = 0.02454654686152935 
Loss after step 700 = 0.002366532338783145 
Loss after step 800 = 0.062323641031980515 
Loss after step 900 = 0.0016910142730921507 


In [10]:
test_data = torch.tensor([[35], [36], [37], [38], [39]], dtype=torch.float).unsqueeze(0) / 100
test_data.shape

torch.Size([1, 5, 1])

In [11]:
predict, _ = rnn(test_data, None)
predict[0][-1]

tensor(0.7806, grad_fn=<SelectBackward>)